In [1]:
#Model Data Loading

#read features and label data from csv file
import pandas as pd
train_features_final_df = pd.DataFrame(pd.read_csv("train_features_final.csv")).drop('Unnamed: 0', axis=1)
train_targets_final_df = pd.DataFrame(pd.read_csv("train_targets_final.csv")).drop('Unnamed: 0', axis=1)

#Split training dataset into training subset and testing subset
from sklearn.model_selection import train_test_split
X_train,X_valid,y_train,y_valid= train_test_split(train_features_final_df,train_targets_final_df,test_size=0.3,random_state=0)

#check train subset and validation subset
#print(X_train.shape)
#print(y_train.shape)
#print(X_valid.shape)
#print(y_valid.shape)

In [2]:
#Model Building & Compling
from keras.models import Sequential
from keras.layers import Dropout, Flatten, Dense

DogCat_model = Sequential()
DogCat_model.add(Dense(2, input_shape=X_train.shape[1:], activation='softmax'))
DogCat_model.summary()

DogCat_model.compile(loss='categorical_crossentropy', optimizer='rmsprop', metrics=['accuracy'])

Using TensorFlow backend.


_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_1 (Dense)              (None, 2)                 8066      
Total params: 8,066
Trainable params: 8,066
Non-trainable params: 0
_________________________________________________________________


In [3]:
#Model Training

from keras.callbacks import ModelCheckpoint  

checkpointer = ModelCheckpoint(filepath='model.v1.weights.best.hdf5', 
                               verbose=1, save_best_only=True)

DogCat_model.fit(X_train, y_train, 
          validation_data=(X_valid, y_valid),
          epochs=20, batch_size=20, callbacks=[checkpointer], verbose=1)

Train on 17002 samples, validate on 7287 samples
Epoch 1/20
17002/17002 [==============================] - 4s 258us/step - loss: 0.0563 - acc: 0.9924 - val_loss: 0.0396 - val_acc: 0.9963

Epoch 00001: val_loss improved from inf to 0.03961, saving model to model.v1.weights.best.hdf5
Epoch 2/20
17002/17002 [==============================] - 3s 193us/step - loss: 0.0179 - acc: 0.9982 - val_loss: 0.0312 - val_acc: 0.9977

Epoch 00002: val_loss improved from 0.03961 to 0.03116, saving model to model.v1.weights.best.hdf5
Epoch 3/20
17002/17002 [==============================] - 3s 193us/step - loss: 0.0141 - acc: 0.9987 - val_loss: 0.0209 - val_acc: 0.9982

Epoch 00003: val_loss improved from 0.03116 to 0.02088, saving model to model.v1.weights.best.hdf5
Epoch 4/20
17002/17002 [==============================] - 3s 193us/step - loss: 0.0083 - acc: 0.9992 - val_loss: 0.0224 - val_acc: 0.9981

Epoch 00004: val_loss did not improve from 0.02088
Epoch 5/20
17002/17002 [===========================

In [5]:
#Loading the best model
DogCat_model.load_weights('model.v1.weights.best.hdf5')

#read features data from csv file
import pandas as pd
test_features_df = pd.DataFrame(pd.read_csv("test_features.csv")).drop('Unnamed: 0', axis=1)

#generate predictions using features
predicted_vector = DogCat_model.predict(test_features_df)

In [41]:
#function to reduce absolute prediction
def predict_adjust(x):
    epsilon=0.005
    if x>1-epsilon:
        return 1-epsilon
    if x<epsilon:
        return epsilon
    return x

#try on test files
#move all files to a directory names 'unlabeled'
from sklearn.datasets import load_files     
data = load_files('test')
import numpy as np
from keras.utils import np_utils
test_files = np.array(data['filenames'])

#store prediction list into csv files, using adjusted prediction
kaggle_submission=[]
for test_file,prediction in zip(test_files,predicted_vector):
    kaggle_submission.append([test_file.replace('test/unlabeled/','').replace('.jpg',''),predict_adjust(prediction[1])])

import pandas as pd
kaggle_submission_df=pd.DataFrame(kaggle_submission, columns=['id','label'])
kaggle_submission_df.to_csv('submission_v1.csv', index=False)

#kaggle competitions submit -c dogs-vs-cats-redux-kernels-edition -f submission_v1.csv -m "model v1"  